<a href="https://colab.research.google.com/github/Adam-Harris87/advanced-topics/blob/main/fizz_buzz_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solving FizzBuzz with Keras

## Overview
- This notebook provides an example Keras network to solve FizzBuzz.
- This is a supervised classification problem, so we'll train our model on labeled data.


## What is FizzBuzz?
FizzBuzz is a common programming interview problem. Here's the setup.
- Write a program that outputs all the integers from 1 to 100.
- If the number is a multpile of 3, then output the string "Fizz"
- If the number is a multiple of 5, then output the string "Buzz"
- If the number is a multiple of 15, output the string "FizzBuzz"
- For any other number, output that, number itself.

In [1]:
import warnings
warnings.filterwarnings('ignore')

# imports
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.models import Model

2023-05-23 14:40:25.076724: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Specify the number of binary digits. 
NUM_DIGITS = 10

In [3]:
## Setup the training data for 101-1024. 1024 is the highest number countable with 10 binary digits
raw_training_data = np.array(range(101, 2**NUM_DIGITS))

In [4]:
## This is a numpy array of integers.
raw_training_data[0:20]

array([101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113,
       114, 115, 116, 117, 118, 119, 120])

In [5]:
# We need to binary encode our inputs
def binary_encode(i, NUM_DIGITS):
    return np.array([i >> d & 1 for d in range(NUM_DIGITS)])

In [6]:
print(1, binary_encode(1, NUM_DIGITS))
print(2, binary_encode(2, NUM_DIGITS))
print(3, binary_encode(3, NUM_DIGITS))
print(4, binary_encode(4, NUM_DIGITS))


1 [1 0 0 0 0 0 0 0 0 0]
2 [0 1 0 0 0 0 0 0 0 0]
3 [1 1 0 0 0 0 0 0 0 0]
4 [0 0 1 0 0 0 0 0 0 0]


In [7]:
binary_encode(3, NUM_DIGITS)

array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [8]:
print(256, binary_encode(256, NUM_DIGITS))
print(512, binary_encode(512, NUM_DIGITS))
print(1023, binary_encode(1023, NUM_DIGITS))

256 [0 0 0 0 0 0 0 0 1 0]
512 [0 0 0 0 0 0 0 0 0 1]
1023 [1 1 1 1 1 1 1 1 1 1]


In [9]:
encoded_training_data = [binary_encode(i, NUM_DIGITS) for i in raw_training_data]

In [10]:
x_train = np.array(encoded_training_data)
x_train[0]

array([1, 0, 1, 0, 0, 1, 1, 0, 0, 0])

In [11]:
# List out the state space. What are the different states that the data could be in:
# there's only 4 options/states in the FizzBuzz setup
# A number is a multiple of 3 and 5, only 3, only 5, or it ain't
# Now we'll need to one hot encode the training data for y
def fizz_buzz_encode(i):
    if i % 15 == 0:
        return np.array([0, 0, 0, 1]) # encoding for "fizzbuzz"
    elif i % 5 == 0: 
        return np.array([0, 0, 1, 0]) # encoding for "buzz"
    elif i % 3  == 0: 
        return np.array([0, 1, 0, 0]) # encoding for "fizz"
    else:
        return np.array([1, 0, 0, 0]) # encoding for the number output

In [12]:
print("Return only the number", fizz_buzz_encode(1))
print("Multiple of 3", fizz_buzz_encode(3))
print("Multiple of 5", fizz_buzz_encode(5))
print("Return only the number", fizz_buzz_encode(7))
print("Multiple of both 3 and 5", fizz_buzz_encode(15))

Return only the number [1 0 0 0]
Multiple of 3 [0 1 0 0]
Multiple of 5 [0 0 1 0]
Return only the number [1 0 0 0]
Multiple of both 3 and 5 [0 0 0 1]


In [13]:
# y_train is the encoded output. 
# This is our "labeled data" for supervised learning
y_train = np.array([fizz_buzz_encode(i) for i in range(101, 2 ** NUM_DIGITS)])
y_train

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [1, 0, 0, 0],
       ...,
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0]])

In [14]:
# Now let's build our model, add layers, compile, and fit it!
model = Sequential()

2023-05-23 14:44:50.233298: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [15]:
## Add the model's layers. 
model.add(Dense(1000, input_dim=NUM_DIGITS, activation="relu"))
model.add(Dense(1000, activation="relu"))
model.add(Dense(4, activation="softmax"))

In [16]:
# categorical cross entropy is the recommended loss function for 
# metrics is a good
model.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=["accuracy"])

In [37]:
model.fit(x_train, y_train, epochs=10, batch_size=128)

Epoch 1/10
8/8 [==============================] - 0s 7ms/step - loss: 1.1492 - accuracy: 0.5341
Epoch 2/10
8/8 [==============================] - 0s 7ms/step - loss: 1.1489 - accuracy: 0.5341
Epoch 3/10
8/8 [==============================] - 0s 7ms/step - loss: 1.1487 - accuracy: 0.5341
Epoch 4/10
8/8 [==============================] - 0s 7ms/step - loss: 1.1484 - accuracy: 0.5341
Epoch 5/10
8/8 [==============================] - 0s 7ms/step - loss: 1.1482 - accuracy: 0.5341
Epoch 6/10
8/8 [==============================] - 0s 7ms/step - loss: 1.1479 - accuracy: 0.5341
Epoch 7/10
8/8 [==============================] - 0s 7ms/step - loss: 1.1478 - accuracy: 0.5341
Epoch 8/10
8/8 [==============================] - 0s 6ms/step - loss: 1.1477 - accuracy: 0.5341
Epoch 9/10
8/8 [==============================] - 0s 7ms/step - loss: 1.1475 - accuracy: 0.5341
Epoch 10/10
8/8 [==============================] - 0s 6ms/step - loss: 1.1472 - accuracy: 0.5341


In [19]:
# convert fizzbuzz to binary
def fizz_buzz(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]


In [71]:
# Setup x_test  fizzbuzz for prime numbers from 1 to 100
numbers = np.arange(1, 101)
x_test = np.transpose(binary_encode(numbers, NUM_DIGITS))
y_test = model.predict(x_test)

4/4 [==============================] - 0s 3ms/step


In [69]:
y_test

array([[0.3573354 , 0.26810592, 0.2025595 , 0.17199926],
       [0.35477298, 0.26449952, 0.20791318, 0.17281428],
       [0.3873199 , 0.27130502, 0.19151808, 0.14985704],
       [0.35788938, 0.27185252, 0.20142962, 0.16882847],
       [0.39233065, 0.27702615, 0.1813557 , 0.1492876 ],
       [0.39269856, 0.2752226 , 0.18683907, 0.14523987],
       [0.42573515, 0.2773853 , 0.16977231, 0.12710719],
       [0.35069758, 0.26159483, 0.21169512, 0.17601249],
       [0.3920533 , 0.26250592, 0.19138236, 0.15405837],
       [0.39101613, 0.2616855 , 0.19508076, 0.15221769],
       [0.42576304, 0.26227823, 0.1762251 , 0.13573371],
       [0.38692364, 0.26938242, 0.19302267, 0.15067129],
       [0.42753184, 0.26925024, 0.17141166, 0.13180631],
       [0.4277978 , 0.26995352, 0.17416066, 0.12808803],
       [0.46166062, 0.26944557, 0.15669493, 0.11219889],
       [0.35535306, 0.27071467, 0.20492001, 0.1690123 ],
       [0.3907574 , 0.27436504, 0.18946461, 0.14541298],
       [0.38838506, 0.26509595,

In [66]:
# Setup predicted output
predictions = np.vectorize(fizz_buzz, numbers, y_test)
print (predictions)

In [31]:
# correct answers for fizzbuzz on 1-100, these are our actual values
answer = np.array([])
for i in numbers:
    if i % 15 == 0: 
        answer = np.append(answer, "fizzbuzz")
    elif i % 5 == 0: 
        answer = np.append(answer, "buzz")
    elif i % 3 == 0: 
        answer = np.append(answer, "fizz")
    else: answer = np.append(answer, str(i))
print (answer)

['1' '2' 'fizz' '4' 'buzz' 'fizz' '7' '8' 'fizz' 'buzz' '11' 'fizz' '13'
 '14' 'fizzbuzz' '16' '17' 'fizz' '19' 'buzz' 'fizz' '22' '23' 'fizz'
 'buzz' '26' 'fizz' '28' '29' 'fizzbuzz' '31' '32' 'fizz' '34' 'buzz'
 'fizz' '37' '38' 'fizz' 'buzz' '41' 'fizz' '43' '44' 'fizzbuzz' '46' '47'
 'fizz' '49' 'buzz' 'fizz' '52' '53' 'fizz' 'buzz' '56' 'fizz' '58' '59'
 'fizzbuzz' '61' '62' 'fizz' '64' 'buzz' 'fizz' '67' '68' 'fizz' 'buzz'
 '71' 'fizz' '73' '74' 'fizzbuzz' '76' '77' 'fizz' '79' 'buzz' 'fizz' '82'
 '83' 'fizz' 'buzz' '86' 'fizz' '88' '89' 'fizzbuzz' '91' '92' 'fizz' '94'
 'buzz' 'fizz' '97' '98' 'fizz' 'buzz']


In [68]:
# Let's evaluate the model's predictions
evaluate = np.array(answer == predictions)
print (np.count_nonzero(evaluate == True) / 100)

0.0


In [67]:
answer == predictions

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

# Exercise
1. Reassign the NUM_DIGITS to be 5 and re-run the notebook. What do you notice? Take note of the accuracy number.
2. Reassign the NUM_DIGITS to be 11 and re-run the notebook. What do you notice? Take note of the accuracy number. 
3. In the `model.fit` code, change the nb_epoch to be a lower number and then completely rerun the notebook?
4. In the `model.fit` code, change the nb_epoch to be a higher number and then completely rerun the notebook?
5. Experiment with adding extra layers to the model
6. Experiment with changing the first number of layers argument going into `model.add(Dense(1000, activation="relu"))`
7. Experiment with different activation functions (look to the Keras documentation)
8. How good can your model get in classifying out-of-sample numbers for FizzBuzz?